<a href="https://colab.research.google.com/github/gduncan2/CS445_OpticalFlowProj/blob/main/445FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# %pip uninstall cupy cupy-cuda11x -y
# %pip install cupy-cuda120

# %pip install opencv-python
# %pip install cupy-cuda12x
# %pip install scipy
# %pip install matplotlib

import cv2
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np
import glob
import os
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
# from google.colab import drive
# drive.mount('/content/drive')

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
def play_video(video, fps):
    while (video.isOpened()):
        retval, frame = video.read()
        if retval:
            cv2.imshow("Video", frame)
            val = cv2.waitKey(int(1000/fps))
            if val == 27:
                break
        else:
            break
    video.release()
    cv2.destroyAllWindows()

def write_video(filename, frames, fps):
    output = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), fps, (frames[0].shape[1], frames[0].shape[0]), True)
    for frame in frames:
        output.write(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    output.release()

def get_video_frames(video):
    frames = []
    while(video.isOpened()):
        retval, frame = video.read()
        if retval:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        else:
            break
    return frames
def save_half_frames(frames):
    os.makedirs('./outputs/even_frames', exist_ok=True)
    os.makedirs('./outputs/odd_frames', exist_ok=True)

    even_frames = []
    odd_frames = []

    for i, frame in enumerate(frames):
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if i % 2 == 0:
            even_frames.append(frame)
            cv2.imwrite(f"./outputs/even_frames/frame_{i:04d}.png", frame_bgr)
        else:
            odd_frames.append(frame)
            cv2.imwrite(f"./outputs/odd_frames/frame_{i:04d}.png", frame_bgr)

    print(f"Saved {len(even_frames)} even frames and {len(odd_frames)} odd frames.")
def load_saved_frames(folder):
    frame_files = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    loaded_frames = []

    for file in frame_files:
        path = os.path.join(folder, file)
        frame = cv2.imread(path)
        if frame is not None:
            loaded_frames.append(frame)

    print(f"Loaded {len(loaded_frames)} frames from '{folder}'")
    return loaded_frames

In [9]:
def get_frame_pairs(frames: np.ndarray):
    return [(frames[i], frames[i + 1]) for i in range(0, len(frames), 2)]

def process_window_lk(
    window_size_x: int,
    window_size_y: int,
    start_x: int,
    start_y: int,
    frame_tplus1: np.ndarray,
    frame_t: np.ndarray,
    fps: float,
    epsilon: float = 1e-4
) -> cp.ndarray:
    # Calculate the temporal derivative
    # frame_t = cv2.cvtColor(frame_t, cv2.COLOR_BGR2RGB)
    # frame_tplus1 = cv2.cvtColor(frame_tplus1, cv2.COLOR_BGR2RGB)
    H, W, _ = frame_t.shape

    delta_t = 1 / fps
    # Patch extraction and GPU transfer
    patch_t = cp.asarray(frame_t[start_y:start_y+window_size_y, start_x:start_x+window_size_x].astype(np.float32)) / 255
    patch_t1 = cp.asarray(frame_tplus1[start_y:start_y+window_size_y, start_x:start_x+window_size_x].astype(np.float32)) / 255

    # Convert to grayscale
    patch_t_gray  = 0.2989 * patch_t[:,:,0] + 0.5870 * patch_t[:,:,1] + 0.1140 * patch_t[:,:,2]
    patch_t1_gray = 0.2989 * patch_t1[:,:,0] + 0.5870 * patch_t1[:,:,1] + 0.1140 * patch_t1[:,:,2]

    # Compute temporal derivative
    # I_t = (patch_t1_gray - patch_t_gray) / delta_t
    I_t = (patch_t1_gray - patch_t_gray)


    # Placeholder for the spatial derivatives
    I_x = cp.zeros((window_size_x, window_size_y), dtype=cp.float32)
    I_y = cp.zeros((window_size_x, window_size_y), dtype=cp.float32)

    if start_x + window_size_x > W or start_y + window_size_y > H:
        return cp.zeros((2,1), dtype=cp.float32)

    smoothed = cp.asarray(cv2.GaussianBlur(cp.asnumpy(patch_t_gray), (5, 5), sigmaX=1.0, sigmaY=1.0))
    I_x = cp.asarray(cv2.Sobel(cp.asnumpy(smoothed), cv2.CV_32F, 1, 0, ksize=3) * 0.5) # ∂/∂x
    I_y = cp.asarray(cv2.Sobel(cp.asnumpy(smoothed), cv2.CV_32F, 0, 1, ksize=3) * 0.5) # ∂/∂y

    Ix = I_x.flatten()
    Iy = I_y.flatten()
    It = I_t.flatten()

    # The system of equations would look like this
    # [
    #   [∑ I_x^2,   ∑ I_x I_y],
    #   [∑ I_x I_y, ∑ I_y^2]
    # ]

    A00 = cp.sum(Ix * Ix)   # ∑ I_x^2
    A01 = cp.sum(Ix * Iy)   # ∑ I_x I_y
    A11 = cp.sum(Iy * Iy)   # ∑ I_y^2

    B0 = -cp.sum(Ix * It)   # -∑ I_x I_t
    B1 = -cp.sum(Iy * It)   # -∑ I_y I_t

    A = cp.array([[A00, A01],
                  [A01, A11]], dtype=cp.float32)
    B = cp.array([B0, B1],      dtype=cp.float32).reshape(2,1)

    # This is striaght from the video where we check how invertible the matrix is
    # In other words whether the system of equations are well conditioned or not
    # We can think of a system where there is hardly any change, like a patch of texture with no change
    det = A00*A11 - A01*A01
    if det > epsilon:
        uv = cp.linalg.solve(A, B)   # 2×1
    else:
        uv = cp.zeros((2,1), dtype=cp.float32)
    return uv



In [ ]:
!ls "/content/drive/MyDrive/CS445/FinalProj/subset_of_frames"

In [5]:
folder = '/content/drive/MyDrive/CS445/FinalProj/'
frame_files = sorted(glob.glob(folder + 'subset_of_frames/frame_*.png'))
frames = []
for f in frame_files:
    img = cv2.imread(f)
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img_rgb)

if len(frames) == 0:
    raise RuntimeError("No frames loaded — check your folder path or file naming.")
fps = 24
window_w, window_h = 100, 100

frames = cp.array(frames)
T = len(frames)
H, W = frames[0].shape[:2]
nY = H // window_h
nX = W // window_w

flows = cp.zeros((T - 1, nY, nX, 2), dtype=cp.float32)

for t in range(T - 1):
    ft, ftplus1 = frames[t], frames[t + 1]
    for y in range(nY):
        y0 = y * window_h
        for x in range(nX):
            x0 = x * window_w
            uv = process_window_lk(window_w, window_h, start_x=x0, start_y=y0,
                                   frame_tplus1=ftplus1, frame_t=ft, fps=fps)
            flows[t, y, x, 0] = uv[0, 0]
            flows[t, y, x, 1] = uv[1, 0]

T, patch_y, patch_x, uv_shape = flows.shape
print(cp.asnumpy(flows).shape)

RuntimeError: No frames loaded — check your folder path or file naming.

In [6]:
def gradients(frame):
    grad_x = cp.zeros_like(frame)
    grad_y = cp.zeros_like(frame)

    grad_x[:, 1:-1] = frame[:, 2:] - frame[:, :-2]
    grad_y[1:-1, :] = frame[2:, :] - frame[:-2, :]

    return grad_x, grad_y

def point_attentuation(R, maxCorners=100, qualityLevel=0.01, minDistance=10):
    maxR = R.max()
    thresh = qualityLevel * maxR

    # 1. Non-Maximum Suppression
    footprint = cp.ones((minDistance*2+1, minDistance*2+1), dtype=cp.bool_)
    local_max = cupyx.scipy.ndimage.maximum_filter(R, footprint=footprint) == R

    # 2. Apply threshold
    mask = (R >= thresh) & local_max

    # 3. Get (x,y) coords
    ys, xs = cp.nonzero(mask)
    points = cp.stack([xs, ys], axis=1)

    # 4. Sort by R strength descending
    scores = R[ys, xs]
    sort_idx = cp.argsort(scores)[::-1]
    points = points[sort_idx]

    # 5. Take top maxCorners
    if points.shape[0] > maxCorners:
        points = points[:maxCorners]

    return points



def shi_tomansi_points(frame, winsize, numberpts = 100,  qualityLevel=0.01, minDistance=10):
  Scores = []
  grad_x, grad_y = gradients(frame)
  Ixsqsum = (grad_x*grad_x)
  Ixysum = (grad_x*grad_y)
  Iysqsum = (grad_y*grad_y)

  kern = cp.ones((winsize,winsize))
  IXX = cupyx.scipy.ndimage.convolve(Ixsqsum, kern, mode = 'constant')
  IXY = cupyx.scipy.ndimage.convolve(Ixysum, kern, mode = 'constant')
  IYY = cupyx.scipy.ndimage.convolve(Iysqsum, kern, mode = 'constant')

  trace = IXX + IYY
  diff = IXX - IYY
  sqrt_term = cp.sqrt((diff * 0.5)**2 + IXY**2)

  eig1 = trace * 0.5 + sqrt_term
  eig2 = trace * 0.5 - sqrt_term

  R = cp.minimum(eig1, eig2)
  return point_attentuation(R, maxCorners = numberpts, qualityLevel = qualityLevel , minDistance = minDistance)


In [7]:
def computeAffineTransformation(start, dest):
    N = len(start)
    ones = cp.ones((N, 1), dtype=cp.float32)
    s = cp.concatenate([start, ones], axis=1)
    d = dest
    M, _, _, _ = cp.linalg.lstsq(s, d, rcond=None)
    M = M.T
    return M


In [10]:
def OpticalFlowFrameInterp(frame0,frame1):
  h, w = frame0.shape[:2]
  gray0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
  gray_cp = cp.asarray(gray0)
  keypoints = shi_tomansi_points(gray_cp, winsize=20, numberpts=1000)
  points = []
  flows = []
  for pt in keypoints:
      x, y = int(pt[0]), int(pt[1])
      if 0 <= x < w and 0 <= y < h:
          flow = flow = process_window_lk(30, 30, x, y, frame1, frame0, fps=24)

          if not cp.isnan(flow).any():
              dx, dy = flow[0].item(), flow[1].item()
              points.append([x, y])
              flows.append([dx, dy])
  points = np.array(points)
  flows = np.array(flows)

  if len(points) < 10:
      return cv2.addWeighted(frame0, 0.5, frame1, 0.5, 0)

  grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
  points_x = griddata(points, flows[:, 0], (grid_x, grid_y), method='linear', fill_value=0)
  points_y = griddata(points, flows[:, 1], (grid_x, grid_y), method='linear', fill_value=0)

  map_x_0 = (grid_x + 0.5 * points_x).astype(np.float32)
  map_y_0 = (grid_y + 0.5 * points_y).astype(np.float32)

  map_x_1 = (grid_x - 0.5 * points_x).astype(np.float32)
  map_y_1 = (grid_y - 0.5 * points_y).astype(np.float32)

  warp0 = cv2.remap(frame0, map_x_0, map_y_0, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
  warp1 = cv2.remap(frame1, map_x_1, map_y_1, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

  interpolated = cv2.addWeighted(warp0, 0.5, warp1, 0.5, 0)
  return interpolated


In [7]:
def interpolate_full_video(input_path, output_path):
    video = cv2.VideoCapture(input_path)
    frames = get_video_frames(video)
    fps = video.get(cv2.CAP_PROP_FPS)
    print(f"Original frames: {len(frames)}")

    high_fps_frames = []
    for i in range(len(frames) - 1):
        f0 = frames[i]
        f1 = frames[i + 1]
        high_fps_frames.append(f0)
        f_interp = OpticalFlowFrameInterp(f0, f1)
        high_fps_frames.append(f_interp)
        print(f"\rPercent Done: {100 * i / len(frames):.2f}%", end='', flush=True)
    print(f"\rPercent Done: {100}%", end='', flush=True)
    high_fps_frames.append(frames[-1])
    print(f"\nOutput frames: {len(high_fps_frames)}")
    write_video(output_path, high_fps_frames, fps * 2)
    print(f"Written output to {output_path}")

# interpolate_full_video("roadrunner.mp4", "newvid.avi")
interpolate_full_video("media/output_24_fps_nvenc_hevc.mp4", "hogwarts.avi")


KeyboardInterrupt: 

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def interp_pair(i, f0, f1):
    return i, OpticalFlowFrameInterp(f0, f1)

def interpolate_full_video_parallel(input_path, output_path, max_workers=4):
    video   = cv2.VideoCapture(input_path)
    frames  = get_video_frames(video)
    fps     = video.get(cv2.CAP_PROP_FPS)

    pairs = [(i, frames[i], frames[i+1]) for i in range(len(frames)-1)]
    high_fps_frames = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(interp_pair, *p) for p in pairs]
        results = [None]*(len(frames)-1)
        ctr = 0
        for fut in as_completed(futures):
            i, f_interp = fut.result()
            results[i] = f_interp
            ctr += 1
            print(f"\rPercent Done: {100 * ctr / len(frames):.2f}%", end='', flush=True)

    for i, f_interp in enumerate(results):
        high_fps_frames.append(frames[i])
        high_fps_frames.append(f_interp)
    high_fps_frames.append(frames[-1])

    write_video(output_path, high_fps_frames, fps*2)

In [12]:
interpolate_full_video_parallel("media/output_24_fps_nvenc_hevc.mp4", "hogwarts_parallel.avi")

CUDARuntimeError: cudaErrorInvalidValue: invalid argument

In [10]:
original   = cv2.VideoCapture("media/output_24_fps_nvenc_hevc.mp4")
new       = cv2.VideoCapture("hogwarts_parallel.avi")
fps = original.get(cv2.CAP_PROP_FPS)
new_fps = new.get(cv2.CAP_PROP_FPS)
print ("Original %s, New frame generated %s", fps, new_fps)


Original %s, New frame generated %s 24.0 48.0
